# SL in the Extended Chandra Deep Field South in DIA objects and Show Cutouts

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNRS
- member : DESC, rubin-inkind
- creation date : 2025-05-25

- Confluence page : https://rubinobs.atlassian.net/wiki/spaces/LSSTCOM/pages/372867091/System-level+Science+Performance+Verification+Sprint
- slides : https://docs.google.com/presentation/d/1bPjS5NGtiEL2lfjmvP1UYdG_tMEDfZxX26ayhf7MhtY/edit#slide=id.ga2f7fb1a1f_0_70

- where to find the campains
- https://rubinobs.atlassian.net/wiki/spaces/DM/pages/226656354/LSSTComCam+Intermittent+Cumulative+DRP+Runs

- plot Navigator
- https://usdf-rsp.slac.stanford.edu/plot-navigator
- https://usdf-rsp.slac.stanford.edu/plot-navigator/plot/%2Frepo%2Fmain/LSSTComCam%2Fruns%2FDRP%2FDP1%2Fw_2025_05%2FDM-48666/objectTableCore_coaddInputCount_SkyPlot

- Notebooks examples
- https://github.com/lsst-dm/DMTR-401/blob/main/notebooks/test_LVV-T40_T1240.ipynb
- https://github.com/lsst-dm/DMTR-412/blob/tickets/DM-38728/notebooks/test_LVV-T1751_AM1_AM2.ipynb

- author : Sylvie Dagoret-Campagne
- creation date : 2025-05-12
- last update : 2025-05-12
- Redo Visits like here : https://github.com/sylvielsstfr/LSST-Rehearsal2024/blob/main/notebooks/Visits/stat_on_visits_LSSTComCamSim.ipynb
- To find what I did on LSSTComCamSim : https://github.com/sylvielsstfr/LSST-Rehearsal2024/blob/main/notebooks/LightCurves/MultiColor_lightCurves-DMRehearsal2024_01-AuxTel-DZPOnCCD.ipynb
- **Confluence page** : https://rubinobs.atlassian.net/wiki/spaces/LSSTCOM/pages/443613290/Science+Verification+Sprint+Feb+2025


In [ ]:
# Confirm that the version of the Science Pipelines is recent:
! echo $HOSTNAME
! eups list -s | grep lsst_distrib

In [ ]:
from lsst.daf.butler import Butler
import lsst.geom as geom

import lsst.afw.display as afwDisplay

from lsst.geom import SpherePoint, degrees, Point2D, Point2I, Extent2I, Box2I
from lsst.afw.image import ExposureF
from lsst.skymap import PatchInfo, Index2D

In [ ]:
afwDisplay.setDefaultBackend("firefly")

In [ ]:
import os
import gc
import glob
import numpy as np
import pandas as pd
import random

import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia
Gaia.MAIN_GAIA_TABLE = "gaiadr3.gaia_source"  # Reselect Data Release 3, default
Gaia.ROW_LIMIT = 100000
from astropy.visualization import (MinMaxInterval, AsinhStretch, ZScaleInterval, LogStretch, LinearStretch,
                                   ImageNormalize)

In [ ]:
import seaborn as sns
from itertools import cycle, islice

In [ ]:
from astropy.time import Time
from datetime import datetime, timedelta

In [ ]:
import lsst
import lsst.geom as geom
from lsst.geom import Angle
from lsst.geom import SpherePoint
from lsst.geom import AngleUnit

In [ ]:
# For Angle conversion
from astropy.coordinates import Angle
import astropy.units as u
from astropy.coordinates import SkyCoord

In [ ]:
def find_tract_patch_and_display(butler, skymap,ra_deg, dec_deg, band, framecount):
    """
    - butler
    - skymap
    - ra_deg, dec_deg : source
    - band
    - framecount
    """
    # Ouvrir le dépôt
    #butler = Butler(repo)
    #skymap = butler.get("skyMap")
    band_to_color = {"u":"blue","g":"green","r":"red","i":"orange","z":"yellow","y":"purple"}

    datasettype = "objectTable_tract"
    therefs = butler.registry.queryDatasets(datasettype,  collections=collection)

    # Créer le point céleste
    coord = SpherePoint(ra_deg * degrees, dec_deg * degrees)


    found = False
    # loop only on existing tracts
    
    for ref in therefs:
        tract_id = ref.dataId["tract"]
        tractInfo = skymap[tract_id]
        if tractInfo.contains(coord):
            patchInfo = tractInfo.findPatch(coord)
            patch_id = patchInfo.getIndex()  # Tuple (x, y)
            patch_indexseq = patchInfo.getSequentialIndex()
            print(f"Tract: {tract_id}, Patch: {patch_indexseq}")
            found = True
            break
    # Charger l’image coadd correspondante
    dataId = {"tract": tract_id, "patch": patch_indexseq, "band": band,"skymap": skymapName}
    #deepCoadd_calexp
    coadd = butler.get("deepCoadd_calexp", dataId)
    # Récupération du WCS
    wcs = coadd.getWcs()

    datatitle = f"deepcoadd-LSSTComCam : {tract_id} , {patch_indexseq} , {band}"
    print(datatitle)

    # Affichage avec marqueur
    
    display = afwDisplay.Display(frame = framecount)
    display.scale('asinh', 'zscale')
    #afw_display.scale('linear', min=zmin, max=zmax)
    #display.setImageColormap(cmap='grey')
    display.setImageColormap(cmap='Grey_r')
    display.mtv(coadd.getImage(),title=datatitle)

    # Création du point céleste
    pix_point = wcs.skyToPixel(coord)
    x, y = pix_point.x,pix_point.y
    
    col = band_to_color[band] 
    display.dot("+", x, y, size=50, ctype=col)
    

    
    return tract_id, patch_indexseq, band

In [ ]:
def extract_deepCoadd_cutout(
    ra_deg,
    dec_deg,
    tractInfo,
    patchInfo,
    band,
    butler,
    skymapName,
    cutout_size_pixels,
    collection,
):
    """
    Extract a square cutout from a deepCoadd image centered on given sky coordinates.

    Parameters
    ----------
    ra_deg : float
        Right ascension in degrees.
    dec_deg : float
        Declination in degrees.
    tractInfo : lsst.skymap.TractInfo
        Tract information object (from SkyMap).
    patchInfo : lsst.skymap.PatchInfo
        Patch information object (from SkyMap).
    band : str
        Photometric band (e.g., 'i', 'r', 'g').
    butler : lsst.daf.butler.Butler
        Butler instance to access LSST data.
    cutout_size_pixels : int, optional
        Size of the cutout in pixels (square region), default is 200.
    collection : str, optional
        Name of the collection containing deepCoadd data.

    Returns
    -------
    cutout : lsst.afw.image.ExposureF
        The extracted image cutout.
    wcs : lsst.afw.geom.SkyWcs
        World Coordinate System associated with the cutout.
    metadata : lsst.daf.base.PropertyList
        FITS metadata header for the cutout image.
    """

    # Build dataId for the deepCoadd image
    tract = tractInfo.getId()
    patch = patchInfo.getSequentialIndex()
    dataId = dict(tract=tract, patch=patch, band=band, skymap=skymapName)

    # Retrieve the deepCoadd exposure
    exposure = butler.get("deepCoadd", dataId=dataId, collections=collection)
    image_bbox = exposure.getBBox()

    # Convert sky coordinates (RA, Dec) to pixel coordinates using WCS
    coord = SpherePoint(ra_deg, dec_deg, degrees)
    wcs = exposure.getWcs()
    pixel_center = wcs.skyToPixel(coord)

    half_size = cutout_size_pixels // 2

    # Centre du cutout
    center_x = int(pixel_center.getX())
    center_y = int(pixel_center.getY())

    # Coordonnées du coin en bas à gauche
    corner_x = max(center_x - half_size, image_bbox.getMinX())
    corner_y = max(center_y - half_size, image_bbox.getMinY())

    # Ne pas dépasser la taille max de l'image
    corner_x = min(corner_x, image_bbox.getMaxX() - cutout_size_pixels)
    corner_y = min(corner_y, image_bbox.getMaxY() - cutout_size_pixels)

    # Define a square bounding box around the target pixel
    # Création du BBox valide
    corner = Point2I(corner_x, corner_y)
    bbox = Box2I(corner, Extent2I(cutout_size_pixels, cutout_size_pixels))

    # Extract the cutout from the original exposure
    cutout = exposure.Factory(exposure, bbox, deep=True)

    return cutout, cutout.getWcs(), cutout.getMetadata()

In [ ]:
def RetrieveDiaObject_forTarget(butler,center_coord,datasettype,where_clause,radius_cut=50):
    """
    Find the closest DIA objectto the target_coord 

    parameters:
    - butler
    - the coordinate of the target (center of the cone seach)
    - the datasettype name for the DIA object
    - where_clause : which contrain requirements on the tract and patch numbers
    - cut on angluar separation for the returned for the returned object

    Return
    - object Id with minimum separation , 
    - minimum separation (arcec),
    - the table of DIA objects within the radius_cut
    """

    #datasettype =  'forcedSourceOnDiaObjectTable'
    #datasettype =  'diaSourceTable_tract'
    #datasettype = 'goodSeeingDiff_assocSsSrcTable' # bad table
    
    therefs = butler.registry.queryDatasets(datasettype,  collections=collection, where=where_clause)
    listref = list(therefs)
    nref = len(listref)
    print(f"nref = {nref}")
    # assume the target is in a single (tract,patch)
    assert nref == 1

    # loop forcedSourceOnDiaObjectTable (per tract, per patch)
    for count,ref in enumerate(therefs):
        the_id = ref.dataId
        the_tract_id = the_id["tract"] 
        
        assert the_tract_id == TRACTSEL

        # catalog of rubin objects (a pandas Dataframe) inside the tract
        catalog = butler.get(ref)
        nsources = len(catalog)
           

        # extract the (ra,dec) coordinates for all te objects of the rubin-catalog
        ra_cat = catalog["ra"].values
        dec_cat = catalog["dec"].values
        # coordinates for all rubin-catalog points
        catalog_coords = SkyCoord(ra=ra_cat*u.deg, dec=dec_cat*u.deg)

        # Angular distance to target
        distances_arcsec = center_coord.separation(catalog_coords).arcsecond

        # add the separation angle to the ctalog
        catalog["sep"] = distances_arcsec


        # closest object from the target
        sepMin = distances_arcsec.min() 
        sepMin_idx = np.where(distances_arcsec == sepMin)[0][0]
    
        closest_obj = catalog[catalog["sep"] <=  sepMin]
                   
        # select a few of these sources to debug the closest candidate
        nearby_obj = catalog[distances_arcsec < radius_cut]
        
        return closest_obj, sepMin, nearby_obj

      

In [ ]:
# https://pipelines.lsst.io/modules/lsst.geom/getting-started.html
func_degToRad = lambda x : Angle(x,lsst.geom.degrees).asRadians()

In [ ]:
# Set plotting defaults
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from matplotlib.colors import ListedColormap
from matplotlib import colors
zscale = ZScaleInterval()

# Set up some plotting defaults:
plt.rcParams.update({'figure.figsize' : (12, 8)})
plt.rcParams.update({'font.size' : 24})
plt.rcParams.update({'axes.linewidth' : 3})
plt.rcParams.update({'axes.labelweight' : 3})
plt.rcParams.update({'axes.titleweight' : 5})
plt.rcParams.update({'ytick.major.width' : 3})
plt.rcParams.update({'ytick.minor.width' : 2})
plt.rcParams.update({'ytick.major.size' : 8})
plt.rcParams.update({'ytick.minor.size' : 5})
plt.rcParams.update({'xtick.major.size' : 8})
plt.rcParams.update({'xtick.minor.size' : 5})
plt.rcParams.update({'xtick.major.width' : 3})
plt.rcParams.update({'xtick.minor.width' : 2})
plt.rcParams.update({'xtick.direction' : 'in'})
plt.rcParams.update({'ytick.direction' : 'in'})


In [ ]:
def nJy_to_ab_mag(f_njy):
    """Convert scalar or array flux in nJy to AB magnitude."""
    f_njy = np.asarray(f_njy)
    mag = np.full_like(f_njy, fill_value=np.nan, dtype=float)
    mask = f_njy > 0
    mag[mask] = -2.5 * np.log10(f_njy[mask]) + 31.4
    return mag


def nJy_err_to_ab_err(f_njy, f_err):
    """Propagate flux error to magnitude error."""
    f_njy = np.asarray(f_njy)
    f_err = np.asarray(f_err)
    mag_err = np.full_like(f_njy, fill_value=np.nan, dtype=float)
    mask = (f_njy > 0) & (f_err > 0)
    mag_err[mask] = (2.5 / np.log(10)) * (f_err[mask] / f_njy[mask])
    return mag_err


In [ ]:
def ab_mag_to_nJy(mag_ab):
    """Convert AB magnitude to flux in nanojanskys."""
    return 10 ** ((31.4 - mag_ab) / 2.5)


## 1) Build the table of SL

- article : https://arxiv.org/pdf/1104.0931

In [ ]:
#15422 44 03:32:38.21 –27:56:53.2 
ra1 = "03:32:38.21 hours"
dec1 = "-27:56:53.2 degrees"
tile1 = 44

#34244 94 03:32:06.45 –27:47:28.6 
ra2 = "03:32:06.45 hours"
dec2 = "-27:47:28.6 degrees"
tile2 = 94
# Je ne trouve pas cette tile ==> FindTileForCutoutGEM
tile2 = 32

#40173 35 03:33:19.45 –27:44:50.0 
ra3 = "03:33:19.45 hours"
dec3 = "-27:44:50.0 degrees"
tile3 = 35

#43242 45 03:31:55.35 –27:43:23.5 
ra4 = "03:31:55.35 hours"
dec4 = "-27:43:23.5 degrees"
tile4 = 45

#46446 47 03:31:35.94 –27:41:48.2 
ra5 = "03:31:35.94 hours"
dec5 = "-27:41:48.2 degrees"
tile5 = 47

#12589 03:31:24.89 −27:58:07.0
ra6 = "03:31:24.89 hours"
dec6 = "-27:58:07.0 degrees"
tile6 = 17

#43797 03:31:31.74 −27:43:00.8 
ra7 = "03:31:31.74 hours"
dec7 = "-27:43:00.8 degrees"
tile7 = 47

#28294 03:31:50.54 −27:50:28.4 
ra8 = "03:31:50.54 hours"
dec8 = "-27:50:28.4 degrees"
tile8 = 33

#36857 03:31:53.24 −27:46:18.9
ra9 = "03:31:53.24 hours"
dec9 = "-27:46:18.9 degrees"
tile9 = 38

#36714 03:32:59.78 −27:46:26.4 
ra10 = "03:32:59.78 hours"
dec10 = "-27:46:26.4 degrees"
tile10 = 37


In [ ]:
lsstcomcam_targets = {}
# high rank
lsstcomcam_targets["ECDFS_G15422"] = {"field_name": "GEMS-15422", "ra": 53.159208333333325, "dec": -27.94811111111111,"tile":tile1}
lsstcomcam_targets["ECDFS_G34244"] = {"field_name": "GEMS-34244", "ra": 53.02687499999999 , "dec": -27.79127777777778,"tile":tile2}
lsstcomcam_targets["ECDFS_G40173"] = {"field_name": "GEMS-40173", "ra": 53.33104166666666 , "dec": -27.747222222222224,"tile":tile3}
lsstcomcam_targets["ECDFS_G43242"] = {"field_name": "GEMS-43242", "ra": 52.980624999999996 , "dec": -27.72319444444444,"tile":tile4}
lsstcomcam_targets["ECDFS_G46446"] = {"field_name": "GEMS-46446", "ra": 52.89975 , "dec": -27.696722222222224,"tile":tile5}

# low rank
lsstcomcam_targets["ECDFS_G12589"] = {"field_name": "GEMS-12589", "ra": 52.85370833333333, "dec": -27.96861111111111,"tile":tile6}
lsstcomcam_targets["ECDFS_G43797"] = {"field_name": "GEMS-43797", "ra": 52.88224999999999, "dec": -27.71688888888889,"tile":tile7}

lsstcomcam_targets["ECDFS_G28294"] = {"field_name": "GEMS-28294", "ra": 52.960583333333325 , "dec": -27.84122222222222,"tile":tile8}
lsstcomcam_targets["ECDFS_G6857"] = {"field_name": "GEMS-6857", "ra": 52.97183333333333 , "dec": -27.771916666666666,"tile":tile9}
lsstcomcam_targets["ECDFS_G36714"] = {"field_name": "GEMS-36714", "ra": 53.249083333333324, "dec": -27.773999999999997,"tile":tile10}


In [ ]:
df = pd.DataFrame(lsstcomcam_targets).T

In [ ]:
df

In [ ]:
# candidates
key = "ECDFS_G15422"
#key = "ECDFS_G34244"
#key = "ECDFS_G40173"
#key= "ECDFS_G43242"
#key= "ECDFS_G46446"

# unknown
#key = "ECDFS_G12589"
#key = "ECDFS_G43797"
#key = "ECDFS_G28294"
#key = "ECDFS_G6857"
#key = "ECDFS_G36714"

the_target = lsstcomcam_targets[key]
target_ra = the_target["ra"]
target_dec = the_target["dec"]
target_name = the_target["field_name"]

target_title = (
    the_target["field_name"] +  f" (ra,dec) = ({target_ra:.2f},{target_dec:.2f}) "
)
target_point = SpherePoint(target_ra, target_dec, degrees)

## Initialisation of rubin science pipeline
- Check here the collection available : https://rubinobs.atlassian.net/wiki/spaces/DM/pages/226656354/LSSTComCam+Intermittent+Cumulative+DRP+Runs

### Configuration

In [ ]:
# The output repo is tagged with the Jira ticket number "DM-40356":
repo = '/repo/main'
#collection = 'LSSTComCam/runs/DRP/DP1/w_2025_05/DM-48666' # work
#collection = 'LSSTComCam/runs/DRP/DP1/w_2025_06/DM-48810' # work
collection = 'LSSTComCam/runs/DRP/DP1/w_2025_10/DM-49359' # work


# bad : crash collection = 'LSSTComCam/runs/DRP/DP1/w_2025_08/DM-49029'

# bad : collection = "LSSTComCam/runs/DRP/20241101_20241211/w_2024_51/DM-48233"

# not working perhaps because I am using w_2025_10 version
# bad : no ccd visit collection = "LSSTComCam/runs/DRP/DP1/w_2025_14/DM-49864"
# bad : no ccd visit collection = 'LSSTComCam/runs/DRP/DP1/w_2025_15/DM-50050'
# bad : no cce visit collection = 'LSSTComCam/runs/DRP/DP1/w_2025_14/DM-49864'
# bad : no cce visit collection collection = 'LSSTComCam/runs/DRP/DP1/w_2025_13/DM-49751'


instrument = "LSSTComCam"
skymapName = "lsst_cells_v1"
where_clause = "instrument = \'" + instrument+ "\'"
collectionStr = collection.replace("/", "_")

In [ ]:
FLAG_DUMP_COLLECTIONS = False
FLAG_DUMP_DATASETS = False
FLAG_DUMP_OBJECTSTABLECOLUMNS = False
FLAG_CUT_OBJECTSMAG = True
FLAG_CUT_OBJECTSSNR = True

In [ ]:
MAGCUT = 24.0
SNRCUT = 5.0

In [ ]:
all_bands = ["u", "g", "r", "i", "z", "y"]
all_bands_colors = ["blue", "green", "red", "orange", "yellow", "purple"]

### Initialisation of Butler

In [ ]:
# Initialize the butler repo:
butler = Butler(repo, collections=collection)
registry = butler.registry

In [ ]:
skymap = butler.get('skyMap', skymap=skymapName, collections=collection)

In [ ]:
camera = butler.get("camera", collections=collection, instrument=instrument)

In [ ]:
# 4. Trouver dans quel tract/patch se trouve la coordonnée
ECDFS_tract_info = skymap.findTract(target_point)

In [ ]:
print('Tract number for SL target in ECDFS ::',ECDFS_tract_info.getId())

In [ ]:
TRACTSEL = ECDFS_tract_info.getId()

In [ ]:
print(f"TRACT : {TRACTSEL}")

In [ ]:
# Check here the collections available
if FLAG_DUMP_COLLECTIONS:
    for _ in sorted(registry.queryCollections(expression=instrument + "/*")):
        if "/calib/" not in _ and "u/" not in _:
            print(_)

In [ ]:
if FLAG_DUMP_DATASETS:
    for datasetType in registry.queryDatasetTypes():
        if registry.queryDatasets(datasetType, collections=collection_validation).any(
            execute=False, exact=False
        ):
            # Limit search results to the data products
            if (
                ("_config" not in datasetType.name)
                and ("_log" not in datasetType.name)
                and ("_metadata" not in datasetType.name)
                and ("_resource_usage" not in datasetType.name)
                and ("Plot" not in datasetType.name)
                and ("Metric" not in datasetType.name)
                and ("metric" not in datasetType.name)
                and (("Table" in datasetType.name) or ("Zeropointp" in datasetType.name) or ("fgcm" in datasetType.name) or ("transm" in datasetType.name) or ("Transm" in datasetType.name)
                or ("source" in datasetType.name) or ("Source" in datasetType.name) or ("object" in datasetType.name) or ("Object" in datasetType.name))
            ):
                
                print(datasetType)

### Get list of Tracts and Patches

- Just to know which tracts are involved in the observations

In [ ]:
datasettype = "objectTable_tract"
therefs = butler.registry.queryDatasets(datasettype,  collections=collection)

In [ ]:
tractsId_list = np.unique([ref.dataId['tract'] for ref in therefs])
tractsId_list = sorted(tractsId_list)
print(tractsId_list)

## Select SL which are in the selected Patch

In [ ]:
ra_target,dec_target = target_ra,target_dec
target_point = SpherePoint(ra_target, dec_target, degrees)
if ECDFS_tract_info.contains(target_point):
    print(ECDFS_tract_info)       

In [ ]:

target_point = SpherePoint(ra_target, dec_target, degrees)
target_coord = SkyCoord(ra=ra_target*u.deg, dec=dec_target*u.deg)

target_name = f"SL :: {key}"


In [ ]:
target_coord

## Find in which patch is the target

In [ ]:
tract_info = ECDFS_tract_info
patch_info = ECDFS_tract_info.findPatch(target_point)
tractNbSel = ECDFS_tract_info.getId()
patchNbSel =  patch_info.getSequentialIndex()
where_clause = f"skymap = '{skymapName}' AND tract = {tractNbSel} AND patch = {patchNbSel}"
#where_clause = f"skymap = '{skymapName}' AND tract = {tractNbSel}"
print(where_clause)

## Define the Stack DM product used for the DIA analysis

In [ ]:
datasettype = 'diaObjectTable_tract'
#datasettype =  'forcedSourceOnDiaObjectTable'
#datasettype =  'diaSourceTable_tract'
#datasettype = 'goodSeeingDiff_assocSsSrcTable' # bad table
subtitle = collectionStr + "_" + datasettype

In [ ]:
closest_obj, sepMin, nearby_obj = RetrieveDiaObject_forTarget(butler,target_coord,datasettype,where_clause)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,4))
nearby_obj["sep"].hist(bins=50,ax=ax,facecolor="b")
closest_obj["sep"].hist(bins=50,range=(0,50),ax=ax,facecolor="r")
ax.set_yscale("log")
ax.set_title("object separation angle")
ax.set_xlabel("separation angle (arcsec)")
plt.show()

In [ ]:
closed_obj_cleaned = closest_obj.dropna(axis=1,how='all')
closed_obj_cleaned.reset_index(level=0, inplace=True)

In [ ]:
closed_obj_cleaned

In [ ]:
closed_obj_cleaned.iloc[0]["ra"]

## Plot the deepCoadds Cutsout

In [ ]:
for ib,band in enumerate(all_bands):
    print(ib,band)
    iframe = ib + 1

    id_name = "diaObjectId" 
    ra_name = "ra"
    dec_name = "dec"
    ndia_name = "nDiaSources"
   
    # object
    df_obj = closed_obj_cleaned.iloc[0]
    #df_obj[id_name] = df_obj[id_name].astype(int)
    name_obj = int(df_obj[id_name])
    ra_obj = float(df_obj[ra_name])
    dec_obj = float(df_obj[dec_name])
    title_obj = f"{target_name} {band} {ra_obj:.3f}, {dec_obj:.3f}"
    ndia_obj = int(df_obj[ndia_name])
    
    # Extract the cutout around the GEM catalog
    cutout, wcs, metadata = extract_deepCoadd_cutout(
        ra_deg=ra_target,
        dec_deg=dec_target,
        tractInfo=tract_info,
        patchInfo=patch_info,
        band=band,
        butler=butler,
        skymapName=skymapName,
        cutout_size_pixels = 100,
        collection = collection,
    )

    # Optionnel : nom unique si plusieurs affichages
    display = afwDisplay.Display(frame=iframe)
    display.scale("asinh", "zscale")
    # Affiche le cutout
    display.mtv(cutout.image, title=title_obj)

    # Show the object
    wcs = cutout.getWcs()
    coord_obj = SpherePoint(ra_obj, dec_obj, degrees)
    pixel_obj = wcs.skyToPixel(coord_obj)  # donne un Point2D (x, y)

    display.dot("o", pixel_obj.getX(), pixel_obj.getY(), size=10, ctype=all_bands_colors[ib])

    coord_target = SpherePoint(ra_target, dec_target, degrees)
    pixel_target = wcs.skyToPixel(coord_target)  # donne un Point2D (x, y)
    display.dot("+", pixel_target.getX(), pixel_target.getY(), size=10, ctype=all_bands_colors[ib])

   

In [ ]:

#display.erase()